In [2]:
import pandas as pd

%load_ext lab_black

In [3]:
df = pd.read_csv("purchase_data.csv")

In [4]:
# player count

total_players = pd.DataFrame({df.SN.nunique()})
total_players.columns = ["Total Players"]
total_players

,Total Players
0,576


In [5]:
# purchasing Analysis (Total)

purchasing_analysis = pd.DataFrame(
    {
        "Number of Unique Items": [df["Item ID"].nunique()],
        "Average Price": "$" + format(df["Price"].mean(), ".2f"),
        "Number of Purchases": [df["SN"].count()],
        "Total Revenue": "$" + format(df["Price"].sum(), ".2f"),
    }
)

purchasing_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


In [20]:
# Gender Demographics

# Column index 0
g_count = df.groupby(df["Gender"])["SN"].nunique().sort_values(ascending=False)


# Column Index 1
total_count = df["SN"].nunique()


def gender_per(e):
    return e / total_count


percent_of_players = [gender_per(e) for e in g_count]


def per_stylized(e):
    return str(format(e * 100, ".2f")) + "%"


per_players = [per_stylized(e) for e in percent_of_players]


gender_dem = pd.DataFrame({"Total Count": g_count, "Percent of Players": per_players})
gender_dem

,Total Count,Percent of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [33]:
# Purchase Analysis (GENDER)

purchase_count = df.groupby(df["Gender"])["Purchase ID"].count()

g_avg_price = df.groupby(df["Gender"])["Price"].mean()

g_total_purchase = df.groupby(df["Gender"])["Price"].sum()

g_avg_total_purchase = (g_total_purchase / g_count).map("${:,.2f}".format)


gen_pur = pd.DataFrame(
    {
        "Purchase Count": purchase_count,
        "Average Purchase Price": g_avg_price,
        "Total Purchase Value": g_total_purchase,
        "Avg Total Purchase per Person": g_avg_total_purchase,
    }
)

gen_pur

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,3.203009,361.94,$4.47
Male,652,3.017853,1967.64,$4.07
Other / Non-Disclosed,15,3.346000,50.19,$4.56


In [27]:
# Age Demographics
# needs work

df_unique = df[["Age", "SN"]].drop_duplicates()

df_unique["age_counts"] = pd.cut(
    df["Age"],
    bins=[0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 200],
    labels=["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"],
)


age_count = df_unique["age_counts"].value_counts()


def players(e):
    return e / total_count


per_players = [str(format(players(e) * 100, ".2f")) + "%" for e in age_count]


df_age = pd.DataFrame({"Total Count": age_count, "Percent of Players": per_players})
df_age

,Total Count,Percent of Players
20-24,258,44.79%
15-19,107,18.58%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
10-14,22,3.82%
<10,17,2.95%
40+,12,2.08%


In [289]:
# Purchasing Analaysis

df["Purchase ID"] = pd.cut(
    df["Age"],
    bins=[0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 200],
    labels=["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"],
)

purchase_count = df["Purchase ID"].value_counts()
purchase_count

# broken here

total_purchase = purchase_count * df["Price"]
total_purchase

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
25-29   NaN
30-34   NaN
35-39   NaN
40+     NaN
<10     NaN
Length: 788, dtype: float64

In [324]:
# Top Spenders
# broken

purchase_count = (
    df.groupby(df["SN"])["Purchase ID"].count().sort_values(ascending=False)
)

total_purchase = df.groupby(df["SN"])["Price"].sum()


spenders_df = pd.DataFrame(
    {
        #     "Purchase Count:" purchase_count,
        #     "Total Purchase Value": total_purchase
    }
)

purchase_count

SN
Lisosia93       5
Iral74          4
Idastidru52     4
Aina42          3
Iri67           3
               ..
Lisossala30     1
Lisossanya98    1
Lisotesta51     1
Lisovynya38     1
Adairialis76    1
Name: Purchase ID, Length: 576, dtype: int64

In [123]:
# Most Popular Items


most_pop = df.groupby(["Item ID", "Item Name", "Price"]).agg(
    {"Price": ["sum", "count"]}
)


most_pop.columns = map("_".join, most_pop)

most_pop = most_pop.rename(
    columns={"Price_sum": "Total Purchase Value", "Price_count": "Purchase Count",}
)


most_pop.sort_values(by="Total Purchase Value", ascending=False)
most_pop

,,,Total Purchase Value,Purchase Count
Item ID,Item Name,Price,,
0,Splinter,1.28,5.12,4
1,Crucifer,3.26,9.78,3
2,Verdict,2.48,14.88,6
3,Phantomlight,2.49,14.94,6
4,Bloodlord's Fetish,1.70,8.50,5
...,...,...,...,...
179,"Wolf, Promise of the Moonwalker",4.48,26.88,6
180,Stormcaller,3.36,3.36,1
181,Reaper's Toll,1.66,8.30,5


In [35]:
# 